<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/friska_test/movies2105.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Wczytanie danych o ocenach i filmach
ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

# Liczba wszystkich ocen
total_ratings = ratings.shape[0]

# Liczba filmów w bazie
total_movies = movies.shape[0]

# Liczba użytkowników
total_users = ratings['userId'].nunique()

# Średnia liczba ocen na użytkownika
avg_ratings_per_user = total_ratings / total_users

# Średnia liczba ocen na film
avg_ratings_per_movie = total_ratings / total_movies

# Rozkład ocen
ratings_distribution = ratings['rating'].value_counts().sort_index()

# Wyświetlenie wyników
print("Liczba wszystkich ocen:", total_ratings)
print("Liczba filmów w bazie:", total_movies)
print("Liczba użytkowników:", total_users)
print("Średnia liczba ocen na użytkownika:", avg_ratings_per_user)
print("Średnia liczba ocen na film:", avg_ratings_per_movie)
print("\nRozkład ocen:")
print(ratings_distribution)


Liczba wszystkich ocen: 100836
Liczba filmów w bazie: 9742
Liczba użytkowników: 610
Średnia liczba ocen na użytkownika: 165.30491803278687
Średnia liczba ocen na film: 10.350646684459043

Rozkład ocen:
rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: count, dtype: int64


In [2]:
# Słownik tytułów filmów
movie_titles = dict(zip(movies['movieId'], movies['title']))

# Obliczenie C i m
C = ratings['rating'].mean()
m = ratings.groupby('movieId')['rating'].agg(['count', 'mean'])
m = m['count'].mean()

# Funkcja bayesian_avg
def bayesian_avg(ratings):
    bayesian_avg = (C * m + ratings.sum()) / (C + ratings.count())
    return bayesian_avg

# Obliczenie bayesian_avg_ratings
bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']

# Połączenie z danymi o filmach
movie_stats = movies.merge(bayesian_avg_ratings, on='movieId')

# Wyświetlenie wyników
print(movie_stats.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  bayesian_avg  
0  Adventure|Animation|Children|Comedy|Fantasy      4.024276  
1                   Adventure|Children|Fantasy      3.645857  
2                               Comedy|Romance      3.708193  
3                         Comedy|Drama|Romance      5.028823  
4                                       Comedy      3.558189  


In [3]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

# Wczytanie danych o ocenach i filmach
ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

# Liczba wszystkich ocen
total_ratings = ratings.shape[0]

# Liczba filmów w bazie
total_movies = movies.shape[0]

# Liczba użytkowników
total_users = ratings['userId'].nunique()

# Średnia liczba ocen na użytkownika
avg_ratings_per_user = total_ratings / total_users

# Średnia liczba ocen na film
avg_ratings_per_movie = total_ratings / total_movies

# Rozkład ocen
ratings_distribution = ratings['rating'].value_counts().sort_index()

# Wyświetlenie wyników
print("Liczba wszystkich ocen:", total_ratings)
print("Liczba filmów w bazie:", total_movies)
print("Liczba użytkowników:", total_users)
print("Średnia liczba ocen na użytkownika:", avg_ratings_per_user)
print("Średnia liczba ocen na film:", avg_ratings_per_movie)
print("\nRozkład ocen:")
print(ratings_distribution)

# Słownik tytułów filmów
movie_titles = dict(zip(movies['movieId'], movies['title']))

# Obliczenie C i m
C = ratings['rating'].mean()
m = ratings.groupby('movieId')['rating'].agg(['count', 'mean'])
m = m['count'].mean()

# Funkcja bayesian_avg
def bayesian_avg(ratings):
    bayesian_avg = (C * m + ratings.sum()) / (C + ratings.count())
    return bayesian_avg

# Obliczenie bayesian_avg_ratings
bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']

# Połączenie z danymi o filmach
movie_stats = movies.merge(bayesian_avg_ratings, on='movieId')

# Wyświetlenie wyników
print(movie_stats.head())

def create_sparse(df):
    M = df['userId'].nunique()
    N = df['movieId'].nunique()
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))
    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))
    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [movie_mapper[i] for i in df['movieId']]
    X = csr_matrix((df["rating"], (user_index, item_index)), shape=(M,N))
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

# Utworzenie macierzy rzadkiej
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_sparse(ratings)

n_total = X.shape[0] * X.shape[1]
n_ratings = X.nnz
sparsity = n_ratings / n_total
sparsity_p = round(sparsity * 100, 2)

print(f"Matrix sparsity: {sparsity_p}%")


Liczba wszystkich ocen: 100836
Liczba filmów w bazie: 9742
Liczba użytkowników: 610
Średnia liczba ocen na użytkownika: 165.30491803278687
Średnia liczba ocen na film: 10.350646684459043

Rozkład ocen:
rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: count, dtype: int64
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  bayesian_avg  
0  Adventure|Animation|Children|Comedy|Fantasy      4.024276  
1                   Adventure|Children|Fantasy      3.645857  
2                               Comedy|Romance      3.708193  
3                         Comedy|Drama|Romance      5.028823  
4                  

In [4]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k, metric='cosine'):
    X = X.T
    neighbour_ids = []
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1, -1)
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    neighbour = neighbour[0][1:]  # Pomijamy pierwszy element, który jest samym filmem
    for n in neighbour:
        neighbour_ids.append(movie_inv_mapper[n])
    return neighbour_ids

# Przykładowe użycie funkcji do znalezienia 10 rekomendacji dla filmu o identyfikatorze 1 (Toy Story)
recommendations = find_similar_movies(1, X, movie_mapper, movie_inv_mapper, k=10)

# Wyświetlenie rekomendacji
for i, movie_id in enumerate(recommendations, 1):
    print(f"{i}. {movie_titles[movie_id]}")


1. Toy Story 2 (1999)
2. Jurassic Park (1993)
3. Independence Day (a.k.a. ID4) (1996)
4. Star Wars: Episode IV - A New Hope (1977)
5. Forrest Gump (1994)
6. Lion King, The (1994)
7. Star Wars: Episode VI - Return of the Jedi (1983)
8. Mission: Impossible (1996)
9. Groundhog Day (1993)
10. Back to the Future (1985)


In [5]:
# Install surprise library
!pip install surprise

# Importing relevant libraries
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Assuming 'ratings' DataFrame is already loaded
# Get minimum and maximum rating from the dataset
min_rating = ratings.rating.min()
max_rating = ratings.rating.max()
reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Define the algorithm
algo = SVD()

# Perform cross-validation
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357257 sha256=adb8631157b31cfae6f56086cd4bc0ee0b21372c71587c26572fb68a07a861a2
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8748  0.8767  0.8787  0.8707  0.8695  0.8741  0.0035  
MAE (testset)     0.6708  0.6728  0.6744  0.6702  0.6693  0.6715  0.0018  
Fit time          1.65    1.63    1.89    2.23    1.64    1.81    0.23    
Test time         0.28    0.14    0.35    0.14    0.25    0.23    0.08    


{'test_rmse': array([0.87484521, 0.87665724, 0.87867681, 0.87072118, 0.8694975 ]),
 'test_mae': array([0.67080424, 0.6728014 , 0.67437825, 0.67020174, 0.66927629]),
 'fit_time': (1.6463375091552734,
  1.6271719932556152,
  1.8852250576019287,
  2.230910539627075,
  1.6404986381530762),
 'test_time': (0.2803940773010254,
  0.14051151275634766,
  0.3480336666107178,
  0.1439213752746582,
  0.2472834587097168)}

In [6]:

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.8703


0.8703286274853377

In [7]:
!pip install scikit-learn
from numpy import mean
from numpy import std

import sklearn.model_selection
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold # StratifiedKFold, RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [8]:
# create dataset
X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1) # prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [10]:

# Utworzenie modelu drzewa decyzyjnego
model = DecisionTreeClassifier()

# Ocena modelu przy użyciu walidacji krzyżowej
scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# Wyświetlenie wyników
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.670 (0.119)
